In [1]:
"""Create parameters.py from a .inp and load it"""
# This cell is facultative, you can use an existing parameters.py

# Directory for .inp file:
path = '/home/yoda/Aron/parameter_studies/Bubble_dynamics_simulation/INP file examples/chem_Otomo2018_without_O.inp'

# import libraries:
import importlib   # for reloading your own files
from termcolor import colored   # for colored error messages
# my own files:
try:
    import inp_data_extractor as inp
except:
    try:
        import Bubble_dynamics_simulation.inp_data_extractor as inp
    except:
        print(colored(f'Error, \'inp_data_extractor.py\' not found', 'red'))
importlib.reload(inp)

# create parameters.py
inp.extract(path)

# load parameters.py
import parameters as par
importlib.reload(par)
print(par.model)

path=/home/yoda/Aron/parameter_studies/Bubble_dynamics_simulation/INP file examples/chem_Otomo2018_without_O.inp
Note, lambda value for specie 'H' is not in data.py: 0.0 is used
Note, lambda value for specie 'NH2' is not in data.py: 0.0 is used
Note, lambda value for specie 'NH' is not in data.py: 0.0 is used
Note, lambda value for specie 'N' is not in data.py: 0.0 is used
Note, lambda value for specie 'NNH' is not in data.py: 0.0 is used
Note, lambda value for specie 'N2H4' is not in data.py: 0.0 is used
Note, lambda value for specie 'N2H3' is not in data.py: 0.0 is used
Note, lambda value for specie 'N2H2' is not in data.py: 0.0 is used
Note, lambda value for specie 'H2NN' is not in data.py: 0.0 is used
Warning, third body 'H2O' is not in species in line 64 (' H2/2.5/ H2O/12/ AR/0.0/ ') in reaction 'H2+M=H+H+M'
Warning, third body 'AR' is not in species in line 64 (' H2/2.5/ H2O/12/ AR/0.0/ ') in reaction 'H2+M=H+H+M'
model: chem_Otomo2018_without_O
File 'parameters.py' succesfully c

In [2]:
"""Libraries"""

# for plotting:
%matplotlib inline
import matplotlib.pyplot as plt
plt.rcParams.update({'font.size': 18})

import numpy as np   # matrices, math
import time   # runtime measurement
from multiprocessing import Pool, cpu_count   # multithreading
import importlib   # reload changes you made
import itertools   # assemble all combinations of control parameters
import json   # convert dictionary to string

# my own file:
already_imported = 'de' in globals()
try:
    import full_bubble_model as de
except:
    try:
        import Bubble_dynamics_simulation.full_bubble_model as de
    except:
        print(colored(f'Error, \'full_bubble_model.py\' not found', 'red'))
if already_imported: importlib.reload(de)   # reload changes you made

model: chem_Otomo2018_without_O
target specie: NH3
excitation: sin_impulse (control parameters: ['p_A', 'freq', 'n'])
enable heat transfer: True	enable evaporation: False	enable reactions: True	enable dissipated energy: True


In [3]:
"""Control parameter ranges and division"""
# a list for each control parameter, containing all the possible values

ranges = dict(
  # Initial conditions:
    # bubble equilibrium radius [m]
    R_E = [1e-6*x for x in np.linspace(1.0, 300.0, 150)], # [um --> m]
    # initial radius / equilibrium radius R_0/R_E [-]
    ratio = [1.0],
  # Ambient parameters:
    # ambient pressure [Pa]
    P_amb = [x * par.atm2Pa for x in [1.0]], # [atm --> Pa]
    # ambient temperature [K]       
    T_inf = [par.absolute_zero + x for x in [20.0]], # [°C --> K]
  # Liquid parameters:
    # water accommodation coefficient [-]
    alfa_M = [0.35],
    #P_v = par.P_v,                    # vapour pressure [Pa]
    #mu_L = par.mu_L,                  # dynamic viscosity [Pa*s]
    # sound speed [m/s]
    c_L = [par.c_L],
    # surfactant (surface tension modfier) [-]
    surfactant = [1.0],
  # Excitation parameters: (excitation_type = no_excitation)
    p_A = [-1e5*x for x in np.linspace(0.5, 4.0, 75)], # pressure amplitude [Pa]
    freq = [40000.0], # frequency [Hz]
    n = [1.0]
)

for key in de.excitation_args:
    if key not in ranges:
        print(colored(f'Error, {key} not in ranges', 'red'))
# print total combinations:
for key in ranges:
    print(f'{key}: {len(ranges[key])}')
total_combinations = f'total combinations: {np.prod([len(ranges[key]) for key in ranges])}'
print(''.join(['_' for i in range(len(total_combinations))]))
print(total_combinations)

R_E: 150
ratio: 1
P_amb: 1
T_inf: 1
alfa_M: 1
c_L: 1
surfactant: 1
p_A: 75
freq: 1
n: 1
_________________________
total combinations: 11250


In [4]:
"""Get all combinations"""
# Make a list, with one dictionary for eachy parameter combinations

start = time.time()
cpars = []
ID = 1
for values in itertools.product(*ranges.values()):
    cpar = dict(zip(ranges.keys(), values))
    cpar['ID'] = ID                      # ID of control parameter (not used during calculation)
    cpar['gases'] = [par.index['H2'], par.index['N2']]    # indexes of species in initial bubble (list of species indexes)
    cpar['fractions'] = [0.75, 0.25]            # molar fractions of species in initial bubble (list of fractions for every gas)
    # Calculate pressure/temperature dependent parameters:
    cpar['mu_L'] = de.Viscosity(cpar['T_inf'])
    cpar['P_v'] = de.VapourPressure(cpar['T_inf'])
    cpars.append(cpar)
    ID += 1

print(f'Assemble cpars: {time.time()-start:.2f} s')
start = time.time()

# Create input dictionary for de.simulate(), a list of dictionaries with cpar and other arguments
kwargs_list = [dict(cpar=cpar, t_int=np.array([0.0, 1.0]), LSODA_timeout=30, Radau_timeout=300) for cpar in cpars]
end = time.time()
print(f'Assemble kwargs_list: {time.time()-start:.2f} s')

Assemble cpars: 0.06 s
Assemble kwargs_list: 0.03 s


In [5]:
"""Save settings as txt"""

# create folder for parameter study results:
file = de.Make_dir('/home/yoda/Aron/parameter_studies/2D_bruteforce/40000_Hz')

# save all settings (full_bubble_model.py, parameters.py, ranges) as txt:
combined_str = f'''parameters settings:
    model = {par.model}
    species = {par.species}
    number of species = {par.K}
    number of reactions = {par.I}

full_bubble_model settings:
    enable_heat_transfer = {de.enable_heat_transfer}
    enable_evaporation = {de.enable_evaporation} 
    enable_reactions = {de.enable_reactions}
    enable_dissipated_energy = {de.enable_dissipated_energy}
    target_specie = \'{de.target_specie}\' # Specie to calculate energy effiqiency
    excitation_type = \'{de.excitation_type}\' # function to calculate pressure excitation

total_combinations = {np.prod([len(ranges[key]) for key in ranges])}
ranges = {json.dumps(ranges, indent=4)}'''

file.write_string(combined_str, 'brutefroce_parameter_sweep_settings')

In [6]:
"""Parameter study, multithread"""
# Runs each combinations in cpars, and saves the results into CSV files
# use Pool(processes=cpu_count()-1) to limit number of threads being used.
# use pool.imap(...) instead of pool.imap_unordered(...) to preserve order in which cpars was made

max_lines = 5000    # maximum length of a CSV
best_energy_efficiency = 1e30

start = time.time()
file.new_file()
with Pool(processes=cpu_count(), maxtasksperchild=100) as pool:
    results = pool.imap_unordered(de.simulate, kwargs_list)

    for data in results:
      # save results:
        if file.lines > max_lines:
            file.close()
            file.new_file()
        data = de.dotdict(data)
        file.write_line(data)
      # print stuff:
        if data.energy_efficiency > 0 and data.energy_efficiency < best_energy_efficiency:
            best_energy_efficiency = data.energy_efficiency
        excitation_params = ''.join([f'{key}={data[key]: <12}; ' for key in de.excitation_args])
        print(f'index: {data.ID: >8}/{len(cpars)};   error_code: {data.error_code: >4};   steps: {data.steps: <8};   runtime: {data.elapsed_time: 6.2f} [s]   |   '+
              f'R_E={1e6*data.R_E: 6.2f} [um]; ratio={data.ratio: 6.2f} [-]; P_amb={1e-5*data.P_amb: 6.2f} [bar]; alfa_M={data.alfa_M: 6.2f} [-]; '+
              f'T_inf={data.T_inf-273.15: 6.2f} [°C]; surfactant={100*data.surfactant: 3.0f} [%]   |   {excitation_params}   |   '+
              f'{de.target_specie} production: {data.energy_efficiency: e} [MJ/kg] (best: {best_energy_efficiency: .1f} [MJ/kg])'+
              '                                                 ', end='\r')
              
file.close()
end = time.time()
elapsed = end - start
print(f'\n\nDONE')
print(f'total time: {(elapsed / 3600): .0f} hours {((elapsed % 3600) / 60): .0f} mins')
print(f'            {elapsed: .2f} [s]   ({(elapsed / len(    cpars)): .2f} [s/run])')

/home/yoda/.local/lib/python3.10/site-packages/scipy/integrate/_ode.py:1348: UserWarning: lsoda: Excess accuracy requested (tolerances too small).
  warnings.warn('{:s}: {:s}'.format(self.__class__.__name__,


 lsoda--  at t (=r1), too much accuracy requested    
       for precision of machine..  see tolsf (=r2)   
      in above,  r1 =  0.1403736665943D-02   r2 =                  NaN
 lsoda--  warning..internal t (=r1) and h (=r2) are  6239   ;   runtime:   5.43 [s]   |   R_E=  1.00 [um]; ratio=  1.00 [-]; P_amb=  1.01 [bar]; alfa_M=  0.35 [-]; T_inf= 20.00 [°C]; surfactant= 100 [%]   |   p_A=-187162.16216216216; freq=40000.0     ; n=1.0         ;    |   NH3 production:  1.527098e+05 [MJ/kg] (best:  8868.8 [MJ/kg])                                                 
       such that in the machine, t + h = t on the next step  
       (h = step size). solver will continue anyway  
      in above,  r1 =  0.5413935376766D-01   r2 =  0.3351620513121D-17
 lsoda--  warning..internal t (=r1) and h (=r2) are  
       such that in the machine, t + h = t on the next step  
       (h = step size). solver will continue anyway  
      in above,  r1 =  0.5413935376766D-01   r2 =  0.3351620513121D-17
 lsoda

/home/yoda/.local/lib/python3.10/site-packages/scipy/integrate/_ode.py:1348: UserWarning: lsoda: Excess accuracy requested (tolerances too small).
  warnings.warn('{:s}: {:s}'.format(self.__class__.__name__,


 lsoda--  at t (=r1), too much accuracy requested    
       for precision of machine..  see tolsf (=r2)   
      in above,  r1 =  0.1115262460736D-02   r2 =                  NaN


/home/yoda/.local/lib/python3.10/site-packages/scipy/integrate/_ivp/common.py:338: RuntimeWarning: overflow encountered in multiply
  new_factor = NUM_JAC_FACTOR_INCREASE * factor[ind]
/home/yoda/.local/lib/python3.10/site-packages/scipy/integrate/_ivp/common.py:360: RuntimeWarning: overflow encountered in multiply
  factor[max_diff < NUM_JAC_DIFF_SMALL * scale] *= NUM_JAC_FACTOR_INCREASE
/home/yoda/.local/lib/python3.10/site-packages/scipy/integrate/_ode.py:1348: UserWarning: lsoda: Excess accuracy requested (tolerances too small).
  warnings.warn('{:s}: {:s}'.format(self.__class__.__name__,


...

capi_return is NULL
Call-back cb_f_in_lsoda__user__routines failed.


capi_return is NULL
Call-back cb_f_in_lsoda__user__routines failed.


 lsoda--  warning..internal t (=r1) and h (=r2) are  3942   ;   runtime:   3.07 [s]   |   R_E= 300.00 [um]; ratio=  1.00 [-]; P_amb=  1.01 [bar]; alfa_M=  0.35 [-]; T_inf= 20.00 [°C]; surfactant= 100 [%]   |   p_A=-68918.91891891892; freq=40000.0     ; n=1.0         ;    |   NH3 production:  4.393503e+24 [MJ/kg] (best:  1011.7 [MJ/kg])                                                  
       such that in the machine, t + h = t on the next step  
       (h = step size). solver will continue anyway  
      in above,  r1 =  0.7087719592447D-02   r2 =  0.3551221239969D-18
 lsoda--  warning..internal t (=r1) and h (=r2) are  
       such that in the machine, t + h = t on the next step  
       (h = step size). solver will continue anyway  
      in above,  r1 =  0.7087719592447D-02   r2 =  0.3551221239969D-18
 lsoda--  warning..internal t (=r1) and h (=r2) are  
       such that in the machine, t + h = t on the next step  
       (h = step size). solver will continue anyway  
      in above

capi_return is NULL
Call-back cb_f_in_lsoda__user__routines failed.


index:    11190/11250;   error_code:    2;   steps: 7067    ;   runtime:  59.30 [s]   |   R_E= 300.00 [um]; ratio=  1.00 [-]; P_amb=  1.01 [bar]; alfa_M=  0.35 [-]; T_inf= 20.00 [°C]; surfactant= 100 [%]   |   p_A=-116216.21621621623; freq=40000.0     ; n=1.0         ;    |   NH3 production:  6.540130e+25 [MJ/kg] (best:  1011.7 [MJ/kg])                                                 

DONE
total time:  2 hours  50 mins
             6605.75 [s]   ( 0.59 [s/run])


In [7]:
# If search is prematurely terminated, close opened file:
file.close()